In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
import tensorflow as tf
from utils.preprocessing import get_gray_images

In [6]:
testing_dir = '../data/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data'
training_dir_base = f'../data/C-NMC_Leukemia/training_data/'

In [7]:
### The code for this lives in notebooks.utils.preprocessing
### Loading the images and grey scaling them
lukemia_images = get_gray_images(training_dir_base+'fold_0/all')
normal_images = get_gray_images(training_dir_base+'fold_0/hem')

In [49]:
### This wll add a reference to the image as the key and the label for that image in the value
### Using this makes the assumption that no two images are identical 

image_dict = {}
for l_image in lukemia_images:
    image_dict[l_image.ref()] = True
for n_image in normal_images:
    image_dict[n_image.ref()] = False

### This will throw and error if there are duplicate images in the data 
assert len(image_dict) == len(lukemia_images) + len(normal_images)

In [65]:
### This is to get the tensor of the image and the label for the image in order since we had to create the labels
image_list = []
image_lab = []
for key, value in image_dict.items():
    image_list.append(key.deref())
    image_lab.append(value)

In [68]:
### Coerse these into np.arrays and the reshape for traning
image_list = np.array(image_list).reshape(len(image_list), 450, 450)
image_lab = np.array(image_lab)

In [71]:
### First model creation
### Note: this is likely going to need tuning
model = tf.keras.Sequential(tf.keras.layers.Flatten(input_shape = [450, 450]))
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [72]:
### Train the model
history = model.fit(image_list, image_lab, epochs=10)

Epoch 1/10
111/111 [==============================] - 15s 136ms/step - loss: 10.4830 - accuracy: 0.3204
Epoch 2/10
 52/111 [=============>................] - ETA: 8s - loss: 10.2895 - accuracy: 0.3329